In [ ]:
import datetime
import json
import requests
import time
from collections import defaultdict
import pickle

from pymongo import MongoClient, InsertOne, DeleteOne, ReplaceOne

In [ ]:

#subreddits to pull
target_list = ['IncelTears','The_Donald','slatestarcodex','TrollXChromosomes', 'politics','mylittlepony']

# Generates a list of date time needed from START to END
start = datetime.datetime.strptime("01-01-2015", "%d-%m-%Y")
end = datetime.datetime.strptime("01-01-2019", "%d-%m-%Y")
date_generated = [
    start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]

post_dict = defaultdict(list)

#Connect to Mongo
client = MongoClient()
db = client["reddit"]
test_collection = db.get_collection('test_insert')
titles_collection = db.get_collection('titles')

In [ ]:
# Loops through each day
for i, date, in enumerate(date_generated):
    print("Working {}".format(date))
    # time.sleep(1)
    if i == len(date_generated) - 1:
        continue

    # Builds link
    start_date = str(int(date_generated[i].timestamp()))
    end_date = str(int(date_generated[i + 1].timestamp()))
    link = 'https://api.pushshift.io/reddit/submission/search/?after=' + start_date + \
        '&before=' + end_date + '&sort_type=score&sort=desc&subreddit=' + target_list[5]

    # Request data from api
    r = requests.get(link)

    # Pulls data from link into json object
    data = json.loads(r.text)
    # Pulls titles from each submission for that days
    for post in data['data']: #checks to make sure we don't already have the post
        if titles_collection.find({'id':post['id']}).count() > 0:
            print ('dupe')
        else:
            titles_collection.insert_one(post)
